In [89]:
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [90]:
dataset = pandas.read_csv("/Users/dependrapathak/train.csv")
dataset.fillna(dataset.mean(axis=0), inplace= True)
dataset.head()

,Id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Ret_175,Ret_176,Ret_177,Ret_178,Ret_179,Ret_180,Ret_PlusOne,Ret_PlusTwo,Weight_Intraday,Weight_Daily
0,1,3.59025,-0.117558,0.558392,0.405572,8.0,0.430972,75751,0.2254,11.0,...,-0.002688,0.002246,-0.000838,-6.953224e-04,0.000003,-0.001974,-0.019512,0.028846,1.251508e+06,1.564385e+06
1,2,3.59025,-0.117558,0.558392,0.405572,3.0,0.388896,17369,0.0166,13.0,...,-0.000129,0.000123,0.000248,3.315418e-07,0.000003,0.000027,-0.002939,-0.010253,1.733950e+06,2.167438e+06
2,3,3.59025,-0.696727,0.739591,-0.167928,9.0,0.471947,8277,0.3650,9.0,...,-0.000524,-0.000394,0.000116,5.322557e-04,0.000274,0.000784,-0.024791,0.015711,1.529197e+06,1.911497e+06
3,4,3.59025,-0.694350,1.568248,0.479073,5.0,0.120653,22508,0.2654,13.0,...,0.000346,-0.000090,0.000288,-1.281102e-04,0.000074,0.000341,-0.005680,-0.002190,1.711569e+06,2.139462e+06
4,5,6.00000,-1.736489,2.765531,1.245280,7.0,4.866985,22423,0.2138,13.0,...,-0.001235,0.000027,0.002449,8.619882e-06,0.001209,-0.000004,0.036104,-0.026552,1.267270e+06,1.584088e+06


In [91]:
X = dataset.iloc[:,1:27]
#skipping Id in input
Y = dataset.iloc[:,:85]

In [92]:
dataset.values

array([[ 1.00000000e+00,  3.59024974e+00, -1.17557876e-01, ...,
         2.88463645e-02,  1.25150797e+06,  1.56438496e+06],
       [ 2.00000000e+00,  3.59024974e+00, -1.17557876e-01, ...,
        -1.02532062e-02,  1.73395035e+06,  2.16743794e+06],
       [ 3.00000000e+00,  3.59024974e+00, -6.96726938e-01, ...,
         1.57107386e-02,  1.52919738e+06,  1.91149673e+06],
       ...,
       [ 3.99980000e+04,  3.59024974e+00, -1.17557876e-01, ...,
        -6.53859238e-03,  1.81282360e+06,  2.26602950e+06],
       [ 3.99990000e+04,  3.59024974e+00, -2.14569960e-02, ...,
        -1.15718202e-02,  1.50791752e+06,  1.88489689e+06],
       [ 4.00000000e+04,  3.59024974e+00,  1.09386811e+00, ...,
        -2.43454814e-02,  1.53538946e+06,  1.91923682e+06]])

In [93]:
#Normalization
X.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,Ret_MinusTwo
0,3.59025,-0.117558,0.558392,0.405572,8.0,0.430972,75751,0.2254,11.0,4.744703,...,-0.549725,0.803059,-0.925463,2.0,0.605593,-0.489492,0.799833,-1.209290,-0.329675,0.055275
1,3.59025,-0.117558,0.558392,0.405572,3.0,0.388896,17369,0.0166,13.0,4.744703,...,-0.949891,0.917958,-0.897067,9.0,0.886205,-0.151805,1.239856,0.953424,-0.709462,0.009748
2,3.59025,-0.696727,0.739591,-0.167928,9.0,0.471947,8277,0.3650,9.0,5.000000,...,-1.132426,1.799813,-1.603488,3.0,0.605593,0.389061,1.728096,-1.798090,-1.019370,0.003077
3,3.59025,-0.694350,1.568248,0.479073,5.0,0.120653,22508,0.2654,13.0,5.000000,...,-1.131213,1.565036,-1.205438,7.0,1.148738,-2.440799,1.551425,-1.788725,-0.329675,0.000984
4,6.00000,-1.736489,2.765531,1.245280,7.0,4.866985,22423,0.2138,13.0,4.000000,...,6.317498,-3.208872,-1.102189,7.0,3.381040,1.732708,1.965189,-5.993092,3.219820,-0.018224


In [94]:
from sklearn import preprocessing

In [95]:
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pandas.DataFrame(x_scaled)

In [96]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.287806,0.436332,0.567033,0.696490,0.777778,0.100957,0.757744,0.606982,0.305556,0.748941,...,0.200456,0.541775,0.564869,0.000,0.229203,0.657678,0.767559,0.732552,0.265723,0.413588
1,0.287806,0.436332,0.567033,0.696490,0.222222,0.097851,0.171126,0.019144,0.361111,0.748941,...,0.161597,0.551262,0.571644,0.875,0.259533,0.699343,0.809666,0.887376,0.201616,0.381757
2,0.287806,0.360283,0.586784,0.628165,0.888889,0.103982,0.079771,1.000000,0.250000,0.800000,...,0.143871,0.624080,0.403093,0.125,0.229203,0.766076,0.856386,0.690401,0.149304,0.377094
3,0.287806,0.360595,0.677112,0.705246,0.444444,0.078049,0.222763,0.719595,0.361111,0.800000,...,0.143989,0.604694,0.498067,0.625,0.287909,0.416922,0.839480,0.691071,0.265723,0.375630
4,0.555556,0.223753,0.807621,0.796529,0.666667,0.428422,0.221909,0.574324,0.361111,0.600000,...,0.867316,0.210495,0.522702,0.625,0.529186,0.931858,0.879074,0.390090,0.864874,0.362201


In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Dense(26, input_dim=26, kernel_initializer='normal', activation='relu'))
    model.add(Dense(85, kernel_initializer='normal'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
seed = 7
np.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))